## Load Data with Panda

## Generator (Dual Boost Performance)

## Left/Right Camera

## Match and Check Data Pair between Y and X

## Preprocess Pictures
### Color to YUV 
### Transit ?
### Control Angle?



## CNN Structure


## Out put Result

In [155]:
import numpy as np
import cv2
import csv
import json
import os.path
import pickle
from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution2D, Flatten, Lambda, Dropout
from keras.optimizers import Adam
import tensorflow as tf
import math
import matplotlib.pyplot as plt

In [156]:
import pandas as pd
data_original = pd.read_csv('driving_log.csv', header = None)
data_original.columns = ["center","left","right","steer_angle","brake","throttle","speed"]
print (data_original)

                                      center  \
0     IMG/center_2016_12_01_13_30_48_287.jpg   
1     IMG/center_2016_12_01_13_30_48_404.jpg   
2     IMG/center_2016_12_01_13_31_12_937.jpg   
3     IMG/center_2016_12_01_13_31_13_037.jpg   
4     IMG/center_2016_12_01_13_31_13_177.jpg   
5     IMG/center_2016_12_01_13_31_13_279.jpg   
6     IMG/center_2016_12_01_13_31_13_381.jpg   
7     IMG/center_2016_12_01_13_31_13_482.jpg   
8     IMG/center_2016_12_01_13_31_13_584.jpg   
9     IMG/center_2016_12_01_13_31_13_686.jpg   
10    IMG/center_2016_12_01_13_31_13_786.jpg   
11    IMG/center_2016_12_01_13_31_13_890.jpg   
12    IMG/center_2016_12_01_13_31_13_991.jpg   
13    IMG/center_2016_12_01_13_31_14_092.jpg   
14    IMG/center_2016_12_01_13_31_14_194.jpg   
15    IMG/center_2016_12_01_13_31_14_295.jpg   
16    IMG/center_2016_12_01_13_31_14_398.jpg   
17    IMG/center_2016_12_01_13_31_14_500.jpg   
18    IMG/center_2016_12_01_13_31_14_602.jpg   
19    IMG/center_2016_12_01_13_31_14_702

### Debuging Check Data Content 

In [157]:
data_original.iloc[1] 

center         IMG/center_2016_12_01_13_30_48_404.jpg
left             IMG/left_2016_12_01_13_30_48_404.jpg
right           IMG/right_2016_12_01_13_30_48_404.jpg
steer_angle                                         0
brake                                               0
throttle                                            0
speed                                         21.8796
Name: 1, dtype: object

In [158]:
len(data_original)

8036

### Slice Data to Use (Not Necessary)

In [159]:
data = data_original[0:8036]
print (data)

                                      center  \
0     IMG/center_2016_12_01_13_30_48_287.jpg   
1     IMG/center_2016_12_01_13_30_48_404.jpg   
2     IMG/center_2016_12_01_13_31_12_937.jpg   
3     IMG/center_2016_12_01_13_31_13_037.jpg   
4     IMG/center_2016_12_01_13_31_13_177.jpg   
5     IMG/center_2016_12_01_13_31_13_279.jpg   
6     IMG/center_2016_12_01_13_31_13_381.jpg   
7     IMG/center_2016_12_01_13_31_13_482.jpg   
8     IMG/center_2016_12_01_13_31_13_584.jpg   
9     IMG/center_2016_12_01_13_31_13_686.jpg   
10    IMG/center_2016_12_01_13_31_13_786.jpg   
11    IMG/center_2016_12_01_13_31_13_890.jpg   
12    IMG/center_2016_12_01_13_31_13_991.jpg   
13    IMG/center_2016_12_01_13_31_14_092.jpg   
14    IMG/center_2016_12_01_13_31_14_194.jpg   
15    IMG/center_2016_12_01_13_31_14_295.jpg   
16    IMG/center_2016_12_01_13_31_14_398.jpg   
17    IMG/center_2016_12_01_13_31_14_500.jpg   
18    IMG/center_2016_12_01_13_31_14_602.jpg   
19    IMG/center_2016_12_01_13_31_14_702

### Resourse:
#### Building powerful image classification models using very little data
https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

## MODEL

### Function: Transform Image (up/down, left/right) and Steering Angle

In [194]:
rows,cols = 160,320
def Trans_Image(image,steer,trans_range):
    # Set up Translation
    # Move picture left or right randomly from -50(-half trans_range) to 50(half trans_range) pixel
    trans_x = trans_range*np.random.uniform(0,1)-trans_range/2
    # Produce a new steering angle based on previous one, minus or plus -0.2 to 0.2 
    steer_angle_tr_image = steer + trans_x/trans_range*2*.2
    # Move picture up or down randomly from -20 to 20 pixel
    trans_y = 40*np.random.uniform(0,1)-40/2
    
    # Geographical Transformation of Image
    # http://docs.opencv.org/trunk/da/d6e/tutorial_py_geometric_transformations.html
    # Transformation Matrix
    Trans_M = np.float32([[1,0,trans_x],[0,1,trans_y]])
    # Pass into warpAffine function to do transformation
    image_tr = cv2.warpAffine(image,Trans_M,(cols,rows))
    
    return image_tr,steer_angle_tr_image

### Function: Resize and Normalize Image

In [195]:
new_size_col,new_size_row =  36,36

def Resize_Image(image):
    shape = image.shape
    # note: numpy arrays are (row, col)!
    # Cut top 1/5 and buttom 25 pixel of every picture
    image = image[math.floor(shape[0]/5):shape[0]-25, 0:shape[1]]
    # Resize picture to new size using openCV
    image = cv2.resize(image,(new_size_col,new_size_row), interpolation=cv2.INTER_AREA)    
    # Normalize Image from -0.5 to 0.5 
    image = image/255.-.5
    return image

### Function: Flip Image 

In [219]:
def Flip_Image(image,y_steer):

    # Generate random number from 0 and 1 to decide with picture to flip
    ind_flip = np.random.randint(2)
    # Random flip image and also steering angle
    if ind_flip==0:
        flip_image = cv2.flip(image,1)
        flip_y_steer = -y_steer
    else:
        flip_image = image
        flip_y_steer = y_steer
        
    return flip_image, flip_y_steer

### Process Image Randomly (Left/Right/Center Camera)

In [220]:
def preprocess_image_file_train(line_data):
    
    # Random pick number from 0 to 2
    i_lrc = np.random.randint(3)
    # Three random numbers will output three different camera angle's picture path in Panda dataframe
    # Three random numbers also determine how much steering angle will be adjusted for different camera angles
    if (i_lrc == 0):
        path_file = line_data['left'][0].strip()
        shift_ang = .25
    if (i_lrc == 1):
        path_file = line_data['center'][0].strip()
        shift_ang = 0.
    if (i_lrc == 2):
        path_file = line_data['right'][0].strip()
        shift_ang = -.25
        
    # The output steering angle will be the original angle for centure camera plus adjusted angle 
    y_steer = line_data['steer_angle'][0] + shift_ang
    
    # Read image from the picture path generated before
    image = cv2.imread(path_file)
    
    # Convert image to YUV from BGR to avoid the shadow affects
    image = cv2.cvtColor(image,cv2.COLOR_BGR2YUV)
    
    # Call "Trans_Image" function, set transformation range to 100
    image,y_steer = Trans_Image(image,y_steer,100) 
    
    # Call "Resize_Image" function 
    image = Resize_Image(image)
    
    # Set image matrix to numpy array
    image = np.array(image)
    
    # Call "Flip_Image" function 
    image,y_steer = Flip_Image(image,y_steer)
    
    return image,y_steer

### Generator (Dual Boost Performance)

In [221]:
def traindata_generator(data,batch_size = 32):
    new_size_col,new_size_row = 36,36
    batch_images = np.zeros((batch_size, new_size_row, new_size_col, 3))
    batch_steering = np.zeros(batch_size)
    while 1:
        for i_batch in range(batch_size):
            i_line = np.random.randint(len(data))
            line_data = data.iloc[[i_line]].reset_index()
            
            keep_pr = 0
            while keep_pr == 0:
                x,y = preprocess_image_file_train(line_data)
                pr_unif = np.random
                if abs(y)<.1:
                    pr_val = np.random.uniform()
                    if pr_val>pr_threshold:
                        keep_pr = 1
                else:
                    keep_pr = 1

            batch_images[i_batch] = x
            batch_steering[i_batch] = y
            
        yield batch_images, batch_steering

In [222]:
traindata_generator (data, batch_size = 32)

<generator object traindata_generator at 0x193925780>

## CNN

In [232]:
from keras.layers import Conv2D, Flatten, MaxPooling2D, Activation, Dropout,ELU

def get_model(time_len=1):
    model = Sequential()
    model.add(Convolution2D(16,  1, 1, subsample=(1, 1), input_shape=(36, 36, 3), activation='elu', border_mode="same"))
    model.add(Convolution2D(16,  1, 1, subsample=(1, 1), activation='elu', border_mode="same"))
    #model.add(Convolution2D(3,  1, 1, subsample=(1, 1), activation='elu', border_mode="same"))
    #model.add(Convolution2D(3,  1, 1, subsample=(1, 1), activation='elu', border_mode="same"))
    model.add(MaxPooling2D(pool_size = (2,2)))
    
    model.add(Dropout(.5))
    #model.add(Activation('elu'))
    
#********************************* 

    model.add(Convolution2D(32, 3, 3, subsample=(1, 1), activation='elu', border_mode="same"))
    model.add(Convolution2D(32, 3, 3, subsample=(1, 1), activation='elu', border_mode="same"))
    #model.add(Convolution2D(32, 3, 3, subsample=(1, 1), activation='elu', border_mode="same"))
    #model.add(Convolution2D(32, 3, 3, subsample=(1, 1), activation='elu', border_mode="same"))
    model.add(MaxPooling2D(pool_size = (2,2)))

    model.add(Dropout(.5))
    #model.add(Activation('elu'))

#*********************************
    model.add(Convolution2D(64, 3, 3, subsample=(1, 1), activation='elu', border_mode="same"))
    model.add(Convolution2D(64, 3, 3, subsample=(1, 1), activation='elu', border_mode="same"))
    #model.add(Convolution2D(64, 3, 3, subsample=(1, 1), activation='elu', border_mode="same"))
    #model.add(Convolution2D(64, 3, 3, subsample=(1, 1), activation='elu', border_mode="same"))
    model.add(MaxPooling2D(pool_size = (2,2)))
    
    model.add(Dropout(.5))
    #model.add(Activation('elu'))
#*********************************
    model.add(Convolution2D(128, 3, 3, subsample=(1, 1), activation='elu', border_mode="same"))
    model.add(Convolution2D(128, 3, 3, subsample=(1, 1), activation='elu', border_mode="same"))
    #model.add(Convolution2D(128, 3, 3, subsample=(1, 1), activation='elu', border_mode="same"))
    #model.add(Convolution2D(128, 3, 3, subsample=(1, 1), activation='elu', border_mode="same"))
    model.add(MaxPooling2D(pool_size = (2,2)))

    model.add(Dropout(.5))
    #model.add(Activation('elu'))

#*********************************
    model.add(Flatten())
    model.add(Dropout(.2))
    #model.add(Activation('elu'))

#*********************************
    model.add(Dense(512))
    #model.add(Dropout(.5))
    model.add(Activation('elu'))
    model.add(Dropout(.5))

    model.add(Dense(256))
    #model.add(Dropout(.5))
    model.add(Activation('elu'))
    model.add(Dropout(.5))
    
    model.add(Dense(128))
    #model.add(Dropout(.5))
    model.add(Activation('elu'))
    model.add(Dropout(.5))
    
    model.add(Dense(64))
    #model.add(Dropout(.5))
    model.add(Activation('elu'))
    model.add(Dropout(.5))
    
    model.add(Dense(16))
    #model.add(Dropout(.5))
    model.add(Activation('elu'))
    model.add(Dropout(.5))
    
    model.add(Dense(1))
    model.add(Activation('linear'))
    #sigmoid

#*********************************

    model.summary()
    model.compile(optimizer="adam", loss="mse", metrics=['accuracy'])

    return model

# Run Model

In [ ]:
model = get_model()
pr_threshold = 0.75 
batch_size = 256
train_r_generator = traindata_generator(data, batch_size)
    
model.fit_generator(train_r_generator, samples_per_epoch = 20000, nb_epoch=3, verbose=1)

    # print('Keep Probability:' + str(pr_threshold))
    
#validation_data = valid_s_generator
#nb_val_samples=val_size

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_426 (Convolution2D (None, 36, 36, 16)    64          convolution2d_input_46[0][0]     
____________________________________________________________________________________________________
convolution2d_427 (Convolution2D (None, 36, 36, 16)    272         convolution2d_426[0][0]          
____________________________________________________________________________________________________
maxpooling2d_149 (MaxPooling2D)  (None, 18, 18, 16)    0           convolution2d_427[0][0]          
____________________________________________________________________________________________________
dropout_312 (Dropout)            (None, 18, 18, 16)    0           maxpooling2d_149[0][0]           
___________________________________________________________________________________________

### Output Model Result to json and h5

In [ ]:
json_string = model.to_json()
with open('model.json','w') as f:
	json.dump(json_string,f,ensure_ascii=False)
model.save_weights('model.h5')

### VGG

In [ ]:
from keras.layers import Conv2D, Flatten, MaxPooling2D, Activation, Dropout,ELU, ZeroPadding2D

def get_model(time_len=1):
    model = Sequential()
    #model.add(ZeroPadding2D((1, 1), batch_input_shape=(64, 64, 3)))
    model.add(Convolution2D(3,  1, 1, subsample=(1, 1), input_shape=(64, 64, 3), activation='relu', border_mode="same"))
# build the rest of the network
    #model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_2'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_2'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_2'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_3'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_2'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_3'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_1'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_2'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_3'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
#*********************************
    model.add(Flatten())

    model.add(Dropout(.2))
    model.add(Activation('elu'))

#*********************************
    model.add(Dense(512))
    model.add(Dropout(.5))
    model.add(Activation('elu'))

    model.add(Dense(64))
    model.add(Dropout(.5))
    model.add(Activation('elu'))

    model.add(Dense(16))
    model.add(Dropout(.5))
    model.add(Activation('elu'))

    model.add(Dense(1))
    model.add(Activation('linear'))

#*********************************



    model.summary()
    model.compile(optimizer="adam", loss="mse", metrics=['accuracy'])

    return model